# Задача 1

Реализовать функции для расчета цены Европейкого или Американкого опциона используя биномиальную модель.

In [20]:
def gen_lattice(S0, u, d, N):
    """generates a binomial lattice
    
    for a given up, down, start value and number of steps (N).
    Resulting lattice has N+1 levels.
    """
    S = [float(S0)]

    for i in range(1, N+1):
        for j in range(0, i+1):
            S.append( S0 * d**j * u**(i-j) )

    return S

def lattice_levels(S):
    return int( round( (m.sqrt(8*len(S)+1)-1)/2 ) )

In [21]:
import sys
print (sys.version)
#if sys.version_info >= (3,4):
#    print( "with enums" )

from enum import Enum
class CallPut(Enum):
    call = 1
    put = 2

class ExerciseStyle(Enum):
    euro = 1
    amer = 2

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


In [22]:
def pv_crr(amerEuro, callPut, S0, K, T, r, sigma, N):
    import math as m
    dt = T / N
    df = m.exp(-r * dt)

    u = m.exp(sigma * m.sqrt(dt))
    d = 1 / u
    p = ( m.exp(r * dt) - d ) / (u - d)

    S = gen_lattice(S0=S0, N=N, u=u, d=d)
    L = N+1 #lattice_levels(S)

    payoff = lambda x: max( 0, x - K ) if CallPut.call == callPut else max( 0, K - x)

    c = len(S)
    for i in range(L):
        c -= 1
        S[c] = payoff(S[c])


    for i in range(N-1, -1, -1):
        for j in range(i, -1, -1):
            c -= 1
            if (ExerciseStyle.euro == amerEuro):
                S[c] = (S[c + i + 1] * p + S[c + i + 2] * (1 - p)) * df
            else:
                S[c] = max(payoff(S[c]), S[c + i + 1] * p + S[c + i + 2] * (1 - p)) * df

    return S[0], S


# parameters
S0 = 100.
T = 1.
r = 0.05
sigma = 0.20
K = 100.
N = 1000

es = ExerciseStyle.euro
pvC, _ = pv_crr(es, CallPut.call, S0=S0, K=K, T=T, r=r, sigma=sigma, N=N)
print( pvC )
pvP, _ = pv_crr(es, CallPut.put, S0=S0, K=K, T=T, r=r, sigma=sigma, N=N)
print( pvP )

es = ExerciseStyle.amer
pvC, _ = pv_crr(es, CallPut.call, S0=S0, K=K, T=T, r=r, sigma=sigma, N=N)
print( pvC )
pvP, _ = pv_crr(es, CallPut.put, S0=S0, K=K, T=T, r=r, sigma=sigma, N=N)
print( pvP )

10.448584103764572


5.571526553833685


10.448584103764572


6.089290810825668


# Задача 2

Напишите код для расчета цены европейского и барьерного опционов для модели Хестона

Сделайте расчет

    * implied волотильности для европейского опциона
    * цены барьерного опциона knock-out 
    

In [3]:
# Параметры оционов
S0 = 80.; K = 85.; T = 1.0
B=110
#
r = 0.05
# Параметры модели
sigma0 = 0.2  # значение волатальности в начальный момент времени
long_term_variance = 0.25  # долгосрочный средний уровень дисперсии
mean_revertion_rate = 0.02 # скорость возврата к среднему
vol_of_vol=0.05  # волатильность волатильности
corr=0.07 # корреляция случайных факторов в модели

Модель задается уравнениями:

$$ dS_t = \mu S_t\,dt + \sqrt{\nu_t} S_t\,dW_1 \,$$

$$ d\nu_t = \theta(\omega - \nu_t)dt + \xi \sqrt{\nu_t}\,dW_2$$

где:

   * $\omega$ - долгосрочный средний уровень (long-term mean) 
   * $\theta$ - скорость возврата к среднему 
   * $\xi$ - волатильность волатильности
   * $W_1 \,$ и $W_2 \,$ винеровские процессы, с корреляцией $\rho \,$.

$$ \left< dW_1\, dW_2 \right> = \rho dt $$

Есть разные схемы дискретизации для процесса $\nu$ в модели Хестона.

Для процесса $\nu$ в модели Хестона будем использовать такой вариант схемы дискретизации:

$$ \nu_{t+dt} = \left( \sqrt{\nu_t} + \frac{1}{2}\xi\sqrt{dt}z_2 \right)^2 + \theta (\omega - \nu_t) dt - \frac{1}{4}\xi^2 dt $$

где $z_2$ случайная величена распределенная по нормальному закону (соответствует процессу $W_2$ )

См. Rouah F D. Euler and Milstein discretization.  http://www.frouah.com/finance%20notes/Euler%20and%20Milstein%20Discretization.pdf
 раздел "2 Milstein Scheme"


Обратите внимание, что для генерации двух процессов нужно сгенерировать две случайные величены $z_1$ и $z_2$, которые бы были скорелированны с коэффицентом $\rho$

In [4]:
import numpy as np
import math as m
import matplotlib.pyplot as plt

In [5]:
N = 360
M = 50000
S = np.zeros((M, N))
v = np.zeros((M, N))

for j in range(M):
    z1 = np.random.standard_normal(N)
    z2 = np.random.standard_normal(N)


    zv = z1
    zs = corr * z1 + (1 - corr ** 2) ** (1 / 2) *z2

    dt = T / N
    S[j][0] = S0
    v[j][0] = sigma0
    for i in range (N-1):
        v[j][i + 1] = (v[j][i]**(1/2) + (1/2)*vol_of_vol*dt**(1/2)*zv[i])**2 + mean_revertion_rate*(long_term_variance - v[j][i])*dt -         (1/4)*vol_of_vol**2 * dt
        S[j][i + 1] = S[j][i] * m.exp((r - (1/2)*v[j][i])*dt + (v[j][i]*dt)**(1/2) * zs[i])

In [6]:
SB = S[np.all(S < B, axis = 1)]

In [7]:
P = S[:, -1]
PB = SB[:, -1]
R = np.sum(v[:, -1]) / M
C = np.sum(m.exp(-r * T) * np.maximum(P - K, 0)) / M
CB = np.sum(m.exp(-r * T) * np.maximum(PB - K, 0)) / M

In [8]:
print('Implied vol ', R)
print("Cost without border ", C)
print("Cost with border ",CB)

Implied vol  0.200993297288509
Cost without border  13.728494072094318
Cost with border  0.5699215045162881
